In [ ]:
DB_NAME = "db_name"
DB_USER = "username"
DB_PASSWORD = "password"
DB_HOST = "localhost"
DB_PORT = "5432"

# 1. Connecting to PostgreSQL and Loading Analysis Tables

This block opens a connection to PostgreSQL and loads the analysis-ready tables into memory. 

The intent is twofold: (1) verify connectivity and (2) fetch the core datasets used throughout the notebook. 

`campaign_events`, `benchmarks`, and `ab_counts` are loaded into pandas DataFrames. Having these in memory allows quick iteration for EDA, plotting, and statistical testing.

In [ ]:
# connecting to Postgres and reading tables
import psycopg2
import pandas as pd

conn = None
try:
    conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT)
    print('Connected to Database.')
except Exception as e:
    print('ERROR connecting to Database — edit credentials above.')
    print(e)
    raise

# reading tables
try:
    benchmarks = pd.read_sql('SELECT * FROM benchmarks ORDER BY n_campaigns DESC LIMIT 1000;', conn)
    ab_counts = pd.read_sql('SELECT * FROM ab_counts ORDER BY impressions DESC;', conn)
    print('Loaded benchmarks.sql and ab_counts.sql')
except Exception as e:
    print('ERROR reading tables. Make sure you executed the SQL files in pgAdmin.')
    print(e)
    if conn: conn.close()
    raise

print()
print("benchmarks rows:", len(benchmarks))
print("ab_counts rows:", len(ab_counts))

# quick peek
benchmarks.head(5)

Connected to Database.
Loaded benchmarks.sql and ab_counts.sql

benchmarks rows: 150
ab_counts rows: 5


/var/folders/nt/4rp641zs5lg7_234f6d1h4_00000gn/T/ipykernel_26213/214217393.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  benchmarks = pd.read_sql('SELECT * FROM benchmarks ORDER BY n_campaigns DESC LIMIT 1000;', conn)
/var/folders/nt/4rp641zs5lg7_234f6d1h4_00000gn/T/ipykernel_26213/214217393.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ab_counts = pd.read_sql('SELECT * FROM ab_counts ORDER BY impressions DESC;', conn)


,market_segment,channel_used,campaign_type,n_campaigns,total_impressions,total_clicks,overall_ctr,avg_conversion_rate,p25_conversion_rate,median_conversion_rate,p75_conversion_rate,avg_roi,median_roi,avg_acquisition_cost,computed_at
0,Fashionistas,Email,Display,1416,7788315,797101,0.102346,0.000746,0.0,0.001,0.001,0.050244,0.051,12540.376412,2025-10-14 21:03:50.570224+00:00
1,Outdoor Adventurers,Email,Email,1394,7659577,775144,0.101199,0.000705,0.0,0.001,0.001,0.049564,0.049,12616.571019,2025-10-14 21:03:50.570224+00:00
2,Tech Enthusiasts,Website,Email,1393,7666225,778054,0.101491,0.000737,0.0,0.001,0.001,0.049133,0.049,12407.205312,2025-10-14 21:03:50.570224+00:00
3,Tech Enthusiasts,Website,Search,1391,7671305,760394,0.099122,0.000720,0.0,0.001,0.001,0.050738,0.051,12660.419842,2025-10-14 21:03:50.570224+00:00
4,Outdoor Adventurers,Website,Search,1391,7708679,779862,0.101167,0.000707,0.0,0.001,0.001,0.049636,0.049,12256.881380,2025-10-14 21:03:50.570224+00:00


In [ ]:
# previewing top variants
ab_counts.head(10)

,variant,conversions,impressions,clicks,n_campaigns,conversion_rate,overall_ctr
0,Search,157625,221415139,22032144,40157,0.000712,0.099506
1,Influencer,158468,220769081,22037657,40169,0.000718,0.099822
2,Email,156882,220144927,21897902,39870,0.000713,0.099470
3,Display,157063,220074756,22030979,39987,0.000714,0.100107
4,Social Media,156885,219056401,21955724,39817,0.000716,0.100229


# 2. Deriving Core KPIs for Exploratory Analysis

This step computes the key performance indicators used throughout the project: **CTR** (clicks ÷ impressions), **conversions** (rate × impressions, if raw conversions are not present), **CPA** (acquisition cost per conversion), and **ROI**. 

Calculating these consistently in one place provides a stable base for both visual exploration and hypothesis testing. Where divisions are involved, we guard against division-by-zero and missing values to avoid distorted metrics.

In [ ]:
# shows expected columns if present
display_cols = ['market_segment','channel_used','campaign_type','n_campaigns','median_conversion_rate','avg_roi','avg_acquisition_cost']
existing = [c for c in display_cols if c in benchmarks.columns]
if len(existing) < 1:
    print("benchmarks table does not contain the expected columns; adjust SQL or inspect table.")
benchmarks[existing].head(12)


,market_segment,channel_used,campaign_type,n_campaigns,median_conversion_rate,avg_roi,avg_acquisition_cost
0,Fashionistas,Email,Display,1416,0.001,0.050244,12540.376412
1,Outdoor Adventurers,Email,Email,1394,0.001,0.049564,12616.571019
2,Tech Enthusiasts,Website,Email,1393,0.001,0.049133,12407.205312
3,Tech Enthusiasts,Website,Search,1391,0.001,0.050738,12660.419842
4,Outdoor Adventurers,Website,Search,1391,0.001,0.049636,12256.881380
5,Foodies,Email,Display,1388,0.001,0.049800,12568.446686
6,Fashionistas,Website,Search,1387,0.001,0.049789,12267.232156
7,Health & Wellness,YouTube,Influencer,1384,0.001,0.050051,12365.846821
8,Tech Enthusiasts,Email,Social Media,1384,0.001,0.050848,12689.687139
9,Outdoor Adventurers,Website,Influencer,1383,0.001,0.049742,12591.109183


# 3. Visuals

## `A.Top Channels by Number of Campaigns`



In [3]:
import plotly.express as px

# aggregate by channel
channel_agg = (
    benchmarks.groupby('channel_used', dropna=False)['n_campaigns']
    .sum()
    .reset_index()
    .sort_values('n_campaigns', ascending=False)
    .head(10)
)

color_seq = px.colors.qualitative.Bold + px.colors.qualitative.Pastel + px.colors.qualitative.Safe

fig = px.bar(
    channel_agg,
    x='channel_used',
    y='n_campaigns',
    text='n_campaigns',
    title='Top Channels by Number of Campaigns',
    labels={'channel_used': 'Channel', 'n_campaigns': 'Campaign Count'},
    color='channel_used',
    color_discrete_sequence=color_seq
)

fig.update_traces(
    texttemplate='%{text:,}',
    textposition='outside',
    marker_line_color='black',
    marker_line_width=1.2
)
fig.update_layout(
    xaxis_tickangle=-40,
    uniformtext_minsize=8,
    uniformtext_mode='show',
    margin=dict(t=80, b=120, l=50, r=50),
    yaxis_title='Number of Campaigns',
    showlegend=False,
    height=500
)
fig.show()




This bar chart ranks the marketing channels by the total number of campaigns conducted on each platform.  
The visualization is built by aggregating campaign counts from the `benchmarks` table and sorting them in descending order.  
Each bar represents how heavily each channel — such as **Email**, **Google Ads**, or **YouTube** — was used in overall marketing efforts.

Analyzing campaign volume by channel gives important context before evaluating performance metrics.  
A channel with a high campaign count (e.g., Email) likely reflects either strong engagement history or strategic prioritization by the marketing team.  
However, high activity alone doesn’t imply effectiveness — it simply indicates where most marketing investment has gone.  
This sets the stage for comparing whether that activity actually converts efficiently in later analyses.


## `B.Median Conversion Rate for Top Channels`

In [8]:
top_ch = channel_agg['channel_used'].tolist()
median_conv = (
    benchmarks[benchmarks['channel_used'].isin(top_ch)]
    .groupby('channel_used')['median_conversion_rate']
    .median()
    .reset_index()
    .sort_values('median_conversion_rate', ascending=False)
)

color_seq2 = px.colors.qualitative.Prism + px.colors.qualitative.Vivid

fig2 = px.bar(
    median_conv,
    x='channel_used',
    y='median_conversion_rate',
    text='median_conversion_rate',
    title='Median Conversion Rate by Channel',
    labels={'median_conversion_rate': 'Median Conversion Rate', 'channel_used': 'Channel'},
    color='channel_used',
    color_discrete_sequence=color_seq2
)

fig2.update_traces(
    texttemplate='%{text:.4f}',
    textposition='outside',
    marker_line_color='black',
    marker_line_width=1.2
)
fig2.update_layout(
    xaxis_tickangle=-40,
    margin=dict(t=80, b=120, l=50, r=50),
    yaxis_tickformat='.2%',
    yaxis_title='Median Conversion Rate',
    showlegend=False,
    height=500
)
fig2.show()



This visualization displays the **median conversion rate** for each marketing channel.  
The conversion rate represents the percentage of users who completed a desired action (e.g., purchase, signup, download) after viewing the campaign.  
By taking the *median* rather than the mean, the chart minimizes the impact of extreme outliers and provides a more reliable measure of central tendency across campaigns.

Each bar corresponds to a marketing channel — such as Email, Social Media, or Google Ads — and shows how efficiently campaigns convert impressions into actual outcomes.  
Despite similar campaign counts across channels, even small differences in median conversion rate can translate into meaningful revenue shifts when scaled.

From this visualization, it’s clear that performance is relatively consistent across channels, suggesting that differences in ROI may come more from cost structures or audience targeting rather than pure conversion efficiency.


## `C.Campaigns by Market Segment and Campaign Type`


In [9]:
if 'market_segment' in benchmarks.columns and 'campaign_type' in benchmarks.columns:
    treedata = (
        benchmarks.groupby(['market_segment', 'campaign_type'], dropna=False)['n_campaigns']
        .sum()
        .reset_index()
    )
    fig3 = px.treemap(
        treedata,
        path=['market_segment', 'campaign_type'],
        values='n_campaigns',
        title='Campaigns by Market Segment and Type',
        color='n_campaigns',
        color_continuous_scale='Tealrose',
        hover_data={'n_campaigns': ':,'}
    )
    fig3.update_layout(margin=dict(t=80, l=30, r=30, b=30))
    fig3.show()
else:
    print("Treemap skipped: missing columns 'market_segment' or 'campaign_type'.")



This treemap provides a high-level view of campaign distribution across **market segments** and **campaign types**.  
Each rectangle represents the total number of campaigns targeting a specific audience (e.g., *Foodies*, *Health & Wellness*, *Tech Enthusiasts*) and delivered via a particular campaign type (e.g., *Email*, *Social Media*, *Influencer*, *Display*).

The color gradient indicates the campaign count, helping to quickly identify where marketing efforts are most concentrated.  
Larger, darker boxes represent segments and types with higher activity levels.

By analyzing the chart, we can observe:
- Certain audience groups like *Foodies* and *Outdoor Adventurers* received broad multi-channel coverage.
- Other segments such as *Fashionistas* or *Health & Wellness* appear more concentrated within a few specific campaign types.

This helps decision-makers assess **market coverage balance** — identifying underrepresented segments or potential oversaturation in others.  
It also aids future planning by revealing which combinations of audience and channel receive the most attention and which might benefit from experimentation.


# 4.Preparing Conversion Metrics

This step verifies that the dataset includes both **conversion counts** and **rates**, which are essential for A/B testing.  
If the raw data lacks a `conversions` column, it is estimated as the product of `conversion_rate` and `impressions`.  
A new `conv_rate` field is then derived to validate consistency between counts and rates.  
Finally, the top variants are previewed to confirm that conversion metrics are complete and correctly computed before statistical comparison.


In [10]:
if 'conversions' not in ab_counts.columns:
    if {'conversion_rate', 'impressions'}.issubset(ab_counts.columns):
        ab_counts['conversions'] = (ab_counts['conversion_rate'] * ab_counts['impressions']).round().astype(int)
        print('Estimated conversions column created.')
    else:
        print('No conversions or conversion_rate/impressions columns found.')

ab_counts['conv_rate'] = ab_counts['conversions'] / ab_counts['impressions']
ab_counts[['variant', 'conversions', 'impressions', 'conv_rate']].head(10)


,variant,conversions,impressions,conv_rate
0,Search,157625,221415139,0.000712
1,Influencer,158468,220769081,0.000718
2,Email,156882,220144927,0.000713
3,Display,157063,220074756,0.000714
4,Social Media,156885,219056401,0.000716


# 5. A/B Test

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
import numpy as np
import plotly.graph_objects as go
from math import sqrt

# basic setup
if ab_counts.shape[0] < 2:
    raise SystemExit("Need at least two variants in ab_counts")

rowB = ab_counts.iloc[0]  # treatment (highest impressions)
rowA = ab_counts.iloc[1]  # control (second highest)

conv_A, n_A = int(rowA['conversions']), int(rowA['impressions'])
conv_B, n_B = int(rowB['conversions']), int(rowB['impressions'])

# running a z-test
count = np.array([conv_B, conv_A])
nobs = np.array([n_B, n_A])
nobs = np.where(nobs == 0, 1, nobs)
stat, pval = proportions_ztest(count, nobs, alternative='two-sided')

rate_A, rate_B = conv_A / n_A, conv_B / n_B
abs_lift = rate_B - rate_A

def prop_ci(k, n, z=1.96):
    p = k / n
    se = sqrt(p * (1 - p) / n)
    return max(0, p - z * se), min(1, p + z * se)

ciA_low, ciA_high = prop_ci(conv_A, n_A)
ciB_low, ciB_high = prop_ci(conv_B, n_B)

variants = [str(rowA['variant']), str(rowB['variant'])]
rates = [rate_A, rate_B]

bar_colors = ['#00BFA6', '#1E88E5']  # calm teal + blue
border_color = 'black'

fig = go.Figure()

for i, var in enumerate(variants):
    ci_low = [ciA_low, ciB_low][i]
    ci_high = [ciA_high, ciB_high][i]
    fig.add_trace(
        go.Bar(
            x=[var],
            y=[rates[i]],
            name=var,
            width=0.4,  # thinner bars
            marker=dict(color=bar_colors[i], line=dict(color=border_color, width=1.3)),
            error_y=dict(
                type="data",
                symmetric=False,
                array=[ci_high - rates[i]],
                arrayminus=[rates[i] - ci_low],
                thickness=1.5,
                width=4,
            ),
            text=[f"{rates[i]*100:.3f}%"],
            textposition="outside",
        )
    )

fig.update_layout(
    title_text=f"Conversion Rates: {variants[0]} (Control) vs {variants[1]} (Treatment)",
    title_x=0.5,
    yaxis_tickformat=".2%",
    yaxis_title="Conversion Rate",
    xaxis_title="Variant",
    template="plotly_white",
    showlegend=False,
    margin=dict(t=80, b=100, l=70, r=70),
    yaxis=dict(showgrid=True, zeroline=True, range=[0, max(rates)*1.2]),  # add padding for text
    height=500,
)

fig.show()

print("\n--- A/B Test Results ---")
print(f"Control = {variants[0]}: {conv_A}/{n_A} ({rate_A:.4%})")
print(f"Treatment = {variants[1]}: {conv_B}/{n_B} ({rate_B:.4%})")
print(f"Absolute lift = {abs_lift:.4%}")
print(f"z-stat = {stat:.4f}")
print(f"p-value = {pval:.6f}")


--- A/B Test Results ---
Control = Influencer: 158468/220769081 (0.0718%)
Treatment = Search: 157625/221415139 (0.0712%)
Absolute lift = -0.0006%
z-stat = -2.3217
p-value = 0.020250


## `Interpretation of A/B Test Results`

This visualization and statistical summary represent the outcome of an A/B test comparing two campaign types — **Influencer** (control) and **Search** (treatment) — to evaluate which channel achieves a higher conversion rate.

The bar chart above shows the **conversion rate** for both variants, along with 95% confidence intervals.  
- The **Influencer** campaign achieved a conversion rate of **0.0718%** (158,468 conversions out of ~220.8M impressions).  
- The **Search** campaign achieved a conversion rate of **0.0712%** (157,625 conversions out of ~221.4M impressions).

While both conversion rates appear visually similar, statistical testing helps determine whether this small numerical difference is meaningful.

The **two-proportion z-test** yielded:
- **Absolute lift:** -0.0006% (Search slightly underperformed)
- **z-statistic:** -2.3217  
- **p-value:** 0.02025  

Since the *p-value (0.02)* is **below the 0.05 significance threshold**, we reject the null hypothesis that both variants perform equally.  
However, the direction of the lift is **negative**, meaning the treatment variant (Search) performs slightly worse than the control (Influencer) despite the result being statistically significant.

#### Key Takeaways:
- The difference in performance is statistically significant but **practically negligible**, as both conversion rates are nearly identical.
- Influencer campaigns currently deliver marginally better conversion efficiency under the same exposure conditions.
- The small effect size suggests that further testing or segmentation (e.g., by audience or device) could reveal whether the observed difference holds across subgroups.

In essence, while this test detects a measurable difference, the practical business implication is limited — favoring stability with Influencer campaigns unless cost structures or secondary KPIs justify otherwise.


In [ ]:
import pandas as pd
from pathlib import Path
import os

desktop = Path(os.path.expanduser("~/Desktop"))
output_path = desktop / "campaign_events_for_tableau.csv"

query = """
SELECT
    market_segment,
    channel_used,
    campaign_type,
    impressions,
    clicks,
    conversion_rate,
    roi,
    acquisition_cost
FROM campaign_events
WHERE impressions IS NOT NULL
LIMIT 50000;  -- sample if dataset is huge
"""

campaigns_df = pd.read_sql(query, conn)
campaigns_df.to_csv(output_path, index=False)

print(f"Exported dataset for Tableau: {output_path}")
print("Rows:", len(campaigns_df))
campaigns_df.head(5)